##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Chatbot Quickstart
<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/Gemini_Chatbot_Tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>
---

This notebook provides an example of how to use the **Gemini API to build a chatbot**.  
You will create a conversational AI that generates responses based on user input.


In [ ]:
# Install dependencies
%pip install -U "google-genai>=1.0.0"

## Set up your API key

To run the following cell, store your API key in a Colab Secret named `GOOGLE_API_KEY`.  
If you don't already have an API key, or you're unsure how to create a Colab Secret, see the  
[Authentication](https://github.com/google-gemini/cookbook/blob/a0b506a8f65141cec4eb9143db760c735f652a59/quickstarts/Authentication.ipynb) quickstart for an example.


In [ ]:
# Import Libraries
from google import genai
from google.genai import types
from google.colab import userdata

# Configure your API key using Colab Secrets
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

### **Interacting with the Gemini Chatbot**  
This guide walks you through building a chatbot using the Gemini API, from basics to advanced features. Each section builds on the previous one, helping you understand how to create a more interactive and intelligent chatbot.  

### **Building a Gemini Chatbot**  
1️. **Send a Basic Message** – Get a response from Gemini with a simple prompt.  
2️. **Multi-Turn Conversations** – Break complex queries into steps for clearer answers.  
3️. **Streaming Responses** – Get real-time responses for a more interactive experience.  
4️. **Conversation History** – Maintain context by referring to past messages.  
5️. **Configuring Model Parameters** – Control response behavior with settings like temperature and max tokens.  
6️. **System Instructions** – Customize chatbot behavior by defining specific roles and response styles.

You can modify the model version using the `MODEL_ID` variable. This step-by-step guide aims to help developers build and optimize chatbots with Gemini.

In [ ]:
# Select the Gemini model
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.0-pro-exp-02-05"] {"allow-input": true, "isTemplate": true}

### **Sending a Basic Message**  
The following section demonstrates how to send a simple message to the Gemini model and receive a response. [Learn more](<https://ai.google.dev/gemini-api/docs/text-generation#text-input>).  


In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=["How does AI work?"]
)
print(response.text)

### **Multi-Turn Conversations**  
The following section demonstrates Gemini ability to collect multiple questions and answers in a chat session. This chat format enables users to step incrementally toward answers and to get help with multipart problems
[Learn more](<https://ai.google.dev/gemini-api/docs/text-generation?hl=en#multi-turn-conversations>).

In [ ]:
#create a chat session
chat = client.chats.create(model = MODEL_ID)

# Send the first message
response = chat.send_message("Each orange has 8 slices.")
print(response.text)

# Send a follow-up message
response = chat.send_message("I have 8 oranges. If 4 slices in total are rotten, how many complete oranges can I eat?")
print(response.text)

# Send another follow-up message
response = chat.send_message("I ate 3 oranges. What is the probability that at least one of the slices I ate was rotten?")
print(response.text)

We can use the conversation history using `chat.get_history()` method and print it in a structured format.

In [ ]:
# Display conversation history
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

### **Streaming Responses**

By default, the model generates and returns the complete response only after finishing the entire text generation process. However, streaming allows real-time output using `send_message_stream()`, enabling responses to be processed as they are generated. This enhances responsiveness, making interactions feel faster and more dynamic, especially for longer replies. [Learn more](<https://ai.google.dev/gemini-api/docs/text-generation#streaming-output>).

In [ ]:
# Send a message with streaming response
response = chat.send_message_stream("Each orange has 8 slices.")
for chunk in response:
    print(chunk.text, end="")

# Send a follow-up message with streaming
response = chat.send_message_stream("I have 8 oranges. If 4 slices in total are rotten, how many complete oranges can I eat?")
for chunk in response:
    print(chunk.text, end="")

# Send another follow-up message with streaming
response = chat.send_message_stream("I ate 3 oranges. What is the probability that at least one of the slices I ate was rotten?")
for chunk in response:
    print(chunk.text, end="")

# Display conversation history
for message in chat.get_history():
    print(f"Role: {message.role}", end=": ")
    print(message.parts[0].text)

### **Maintaining Conversation History**  

In multi-turn conversations, keeping track of previous messages helps to provide relevant and context-aware responses. This section shows how to store past exchanges so users can ask follow-up questions without losing context.
[Learn more](<https://ai.google.dev/api/generate-content#method:-models.generatecontent>).

In [ ]:
# Previous conversation history
conversation_history = [
    types.Content(role="user", parts=[types.Part(text="Hello!")]),
    types.Content(role="model", parts=[types.Part(text="Hi there! How can I assist you today?")]),
    types.Content(role="user", parts=[types.Part(text="I have two dogs in my house. How many paws are in my house?")]),
    types.Content(role="model", parts=[types.Part(text="Each dog has 4 paws, so if you have 2 dogs, that makes 8 paws in your house.")]),
]

# Function to interact with Gemini Chatbot
def chat_with_gemini(prompt: str) -> str:
    """
    Generates a response from the Gemini chatbot for a given prompt.
    - Maintains multi-turn conversation history
    - Streams AI-generated responses in real-time
    - Uses proper message formatting
    """
    
    # Append user input to conversation history
    conversation_history.append(types.Content(role="user", parts=[types.Part(text=prompt)]))

    # Create a chat session with history
    chat = client.chats.create(model=MODEL_ID, history=conversation_history)

    response = chat.send_message_stream(message=prompt)  # Stream model response  
    # response = chat.send_message(message=prompt)  # Get full response at once  

    full_response = ""
    print("Gemini:", end=" ", flush=True)
    for chunk in response:
        print(chunk.text, end="", flush=True)  # Streaming output
        full_response += chunk.text

    # Append response to history
    conversation_history.append(types.Content(role="model", parts=[types.Part(text=full_response)]))

    return full_response

# Ask a follow-up question using the history
user_input = "What if I get another dog?"
response = chat_with_gemini(user_input)

### **Configuring Model Parameters**  

When sending a request to the Gemini API, various parameters can be adjusted to control how the model generates responses. If not specified, the model uses default values. Below are key parameters that can be configured:  

- **`stopSequences`** – Defines sequences that stop response generation. The model stops output when encountering these sequences.  
- **`temperature`** – Controls randomness. Higher values make responses more creative, while lower values make them more deterministic (range: 0.0 to 2.0).  
- **`maxOutputTokens`** – Limits the number of tokens in the response.  
- **`topP`** – Adjusts token selection by probability. A lower value makes responses more focused, while a higher value allows more variety.  
- **`topK`** – Limits token selection to the most probable options. A `topK` of 1 chooses the highest-probability token, while higher values allow more diversity.  

The example below demonstrates how to configure these parameters. [Learn more](<https://ai.google.dev/gemini-api/docs/text-generation?hl=en#configuration-parameters>).

In [ ]:
# Configure model parameters for response generation
response = client.models.generate_content(
    model=MODEL_ID,
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,  # Limit response length
        temperature=0.1         # Lower value for more precise responses
    )
)
print(response.text)

### **Using System Instructions**  

System instructions define the model’s behavior for a specific use case, ensuring consistent responses throughout a conversation. This is especially useful for chatbots, as it helps maintain a specific persona, tone, or role without requiring repeated context in every user message.  

The following code configures a chatbot to act as a friendly travel assistant: [Learn more](<https://ai.google.dev/gemini-api/docs/text-generation?hl=en#system-instructions>).

In [ ]:
# Setting up the chatbot with system instructions
response = client.models.generate_content(
    model=MODEL_ID,
    config=types.GenerateContentConfig(
        system_instruction="You are a friendly travel assistant. Suggest travel destinations and give recommendations."
    ),
    contents="Can you suggest a good beach destination?"
)

# Print response
print(response.text)

# Multiple follow-up questions
follow_ups = [
    "What activities can I do there?",
    "When is the best time to visit?",
    "Are there any budget-friendly accommodations?"
]

# Send them one-by-one
for question in follow_ups:
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=question
    )
print(response.text)

## **Next Steps**  

🔹 **[Handling Long Context](https://ai.google.dev/gemini-api/docs/long-context)**  

🔹 **[Build an AI Chat Web App](https://ai.google.dev/gemini-api/tutorials/web-app?lang=python)**  

🔹 **[Prompt Design Strategies](https://ai.google.dev/gemini-api/docs/prompting-strategies?hl=en)**  